In [1]:
import pandas as pd
import json

In [2]:
fantom_txs_filename = "../var/fantom-txs.csv.gz"
txs = pd.read_csv(fantom_txs_filename)

In [7]:
txs.columns

Index(['block_id', 'tx_hash', 'method', 'from', 'to', 'quantity'], dtype='object')

In [10]:
txs.head()

,block_id,tx_hash,method,from,to,quantity
0,56889636,0x403ad523a886c918542d7286b4b99109b7ce8bb100e7...,0x,0xb3A4cea0545A379ccaE4b3Bd8EF61e43Cb180f9e,0x0724420978846F5EEDed4569757f38e833cF109E,999129676456824152
1,56889636,0x892de2d4f5799a77117014447142602e1196bc8bcdc7...,0xb39500a4,0xDC081533B9CaA5FC621be1B5D12Cfc1E0089882E,0x76703141370677Bf62645cBB8D85b57d5176f7A9,0
2,56889636,0x307c214f48a32b9ccd353926ed942cbb602b690e860f...,0x2ed391c4,0xA90c5aA33cB98Ecd530CCB5eb6cB412755926c37,0x0cE6a57209eb32cceC93E14294f3F8fD62cBc364,0
3,56889636,0x4e146877ecb6b79b41c55961d45ae1c720ae77770a91...,0x095ea7b3,0xC9CE22634703Ec7c430d620EC267b3E042D2b881,0xCbCaF8cB8cbeAFA927ECEE0c5C56560F83E9B7D9,0
4,56889636,0xc5d313eee64f7421126e3ddfe69b172875c0f8f28a76...,0xb39500a4,0x24a58f94C1093E689Aeda1E8b632b005603217de,0x76703141370677Bf62645cBB8D85b57d5176f7A9,0


In [27]:
mpx_erc20_address = "0x66eEd5FF1701E6ed8470DC391F05e27B1d0657eb"
mlp_erc20_address = "0xd5c313DE2d33bf36014e6c659F13acE112B80a8E"
vault_address = "0x3CB54f0eB62C371065D739A34a775CC16f46563e"
mlp_manager_address = "0xA3Ea99f8aE06bA0d9A6Cf7618d06AEa4564340E9"
musd_erc20_address = "0xB7209EbCBF71c0ffA1585B4468A11CFfdcDBB9a9"
esmpx_erc20_address = "0xe0f606e6730bE531EeAf42348dE43C2feeD43505"
reward_router_address = "0x20De7f8283D377fA84575A26c9D484Ee40f55877"
reward_reader_address = "0x512F8D4E28EB53A6d036aEDA9C5a4D1De6DBD543"

In [22]:
mpx_txs = txs.query(f'to == "{mpx_erc20_address}"')
len(mpx_txs)

7617

In [33]:
unique_mpx_addresses = set(mpx_txs['from'])
len(unique_mpx_addresses)

2730

In [23]:
mlp_txs = txs.query(f'to == "{mlp_erc20_address}"')
len(mlp_txs)

5

In [24]:
vault_txs = txs.query(f'to == "{vault_address}"')
len(vault_txs)

18

In [25]:
manager_txs = txs.query(f'to == "{mlp_manager_address}"')
len(manager_txs)

5

In [29]:
reward_router_txs = txs.query(f'to == "{reward_router_address}"')
len(reward_router_txs)

46739

In [32]:
unique_reward_addresses = set(reward_router_txs['from'])
len(unique_reward_addresses)

1585

In [37]:
import json
addresses = unique_reward_addresses | unique_mpx_addresses
with open("../var/fantom-addresses.json", "w") as f:
    json.dump(list(addresses), f)
len(addresses)

3010

In [39]:
list(addresses)[0]

'0xDe0610591B31D2f562D39FcF60ca9d79c3b67D8A'

In [41]:
weth_erc20_address = "0x21be370D5312f44cB42ce377BC9b8a0cEF1A4C83"
tracker_staked_mlp_address = "0xa4157E273D88ff16B3d8Df68894e1fd809DbC007" # staked MPX
tracker_staked_mpx_address = "0x49A97680938B4F1f73816d1B70C3Ab801FAd124B" # fee + staked MLP

In [ ]:
def parse_address(item):
    return item.split(",")[2].split("'")[1]

In [57]:
with open("../var/fantom-staked-mlp-results.json", "r") as f:
    mlp_call_results = json.load(f)

len(mlp_call_results)

3010

In [64]:
mlp_balances = {}
for key, value in mlp_call_results.items():
    if value > 0:
        mlp_balances[parse_address(key)] = value

with open("../var/fantom-fsmlp-balances.json", "w") as f:
    json.dump(mlp_balances, f, indent=2)

sum(mlp_balances.values()) / 1e18

440363.1474220085

In [66]:
with open("../var/fantom-staked-mpx-results.json", "r") as f:
    mpx_call_results = json.load(f)

len(mpx_call_results)

3010

In [67]:
mpx_balances = {}
for key, value in mpx_call_results.items():
    if value > 0:
        mpx_balances[parse_address(key)] = value

with open("../var/fantom-smpx-balances.json", "w") as f:
    json.dump(mpx_balances, f, indent=2)

sum(mpx_balances.values()) / 1e18

7331170.913405434